# Bert Sentiment Analysis - TensorFlow

## Import

In [1]:
!pip install tensorflow-gpu

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
# Install missing librairies
!pip install transformers

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datasets import load_dataset, concatenate_datasets
from datetime import datetime
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import load_dataset, concatenate_datasets
from transformers import TFBertModel, AutoTokenizer
from transformers import DataCollatorWithPadding

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, TFBertForSequenceClassification

In [4]:
plt.rcParams["figure.figsize"] = (16, 9)

In [5]:
# 2. Load the STSB dataset: https://huggingface.co/datasets/sentence-transformers/stsb
train_dataset = load_dataset("/kaggle/input/smsa-dataset", split="train")
eval_dataset = load_dataset("/kaggle/input/smsa-dataset", split="validation")
test_dataset = load_dataset("/kaggle/input/smsa-dataset", split="test")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
max_length = 128
batch_size = 32
eval_batch_size = 32

In [8]:
def map_sentiment_to_int(example):
    sentiment_mapping = {
        "positive": 1,
        "negative": 0,
        "neutral": 2  # Tambahkan label lain jika perlu
    }
    # Ganti nilai sentiment dengan integer
    example["sentiment"] = sentiment_mapping[example["sentiment"]]
    return example

train_dataset = train_dataset.map(map_sentiment_to_int)
eval_dataset = eval_dataset.map(map_sentiment_to_int)
test_dataset = test_dataset.map(map_sentiment_to_int)

Map:   0%|          | 0/10932 [00:00<?, ? examples/s]

Map:   0%|          | 0/1259 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [9]:
def tokenize_function(examples):
    # Tokenize sentences and return tensors
    return tokenizer(
        examples["text"],  # Kolom teks
        padding="max_length",  # Pastikan semua input memiliki panjang sama
        truncation=True,
        max_length=max_length,
        return_tensors="np",  # Pastikan format numpy
    )

train_ds = train_dataset.map(tokenize_function, batched=True)
eval_ds = eval_dataset.map(tokenize_function, batched=True)
test_ds = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 4. Prepare dataset for TensorFlow
train_dataset = train_ds.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['sentiment'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataset = eval_ds.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['sentiment'],
    shuffle=False,
    batch_size=eval_batch_size,
    collate_fn=data_collator,
)

Map:   0%|          | 0/10932 [00:00<?, ? examples/s]

Map:   0%|          | 0/1259 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


### Read dataset

In [10]:
# import pandas as pd

# # Load your dataset
# data_path = "/kaggle/input/smsa-dataset/train_preprocess.csv"  # Replace with your file path
# df = pd.read_csv(data_path)

# # Print column names
# print(df.columns)

# # Rename columns for clarity
# df.columns = ['text', 'sentiment']  # Rename the columns appropriately

In [11]:
# # Remove duplicate rows
# df = df.drop_duplicates()

# # Remove duplicate column names if any
# df = df.loc[:, ~df.columns.duplicated()]


# # Drop rows with missing values in any column
# df = df.dropna()

# # Check for empty strings or invalid entries
# df = df[df['text'].str.strip() != '']  # Remove rows with empty 'text' field
# df = df[df['sentiment'].str.strip() != '']  # Remove rows with empty 'sentiment' field

# # Ensure sentiment labels are consistent
# valid_sentiments = {'positive', 'negative', 'neutral'}
# df = df[df['sentiment'].isin(valid_sentiments)]  # Keep only valid labels


In [12]:
# df

In [13]:
# df.to_csv("cleaned_dataset.csv", index=False)

In [14]:
model = TFBertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=3)

tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model

In [16]:
# Hyperparameters
epochs = 5
learning_rate = 5e-5

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics_fn = tf.keras.metrics.SparseCategoricalAccuracy()

In [17]:
!nvidia-smi

Sat Jan 25 14:03:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             31W /  250W |    1285MiB /  16384MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
import tensorflow as tf
import time
import random
from tqdm import tqdm
from transformers import create_optimizer, AdamW
from datetime import timedelta

def format_time(elapsed):
    return str(timedelta(seconds=int(round((elapsed)))))

def train():
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    tf.random.set_seed(seed_val)

    # Prepare optimizer, learning rate scheduler
    steps_per_epoch = len(train_dataset) // batch_size
    num_train_steps = steps_per_epoch * epochs

    # Metrics to track loss
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    val_loss = tf.keras.metrics.Mean(name='val_loss')
    train_accuracy = tf.keras.metrics.Mean(name="train_accuracy")
    val_accuracy = tf.keras.metrics.Mean(name="val_accuracy")

    # Training and validation loops
    training_stats = []
    total_t0 = time.time()

    for epoch_i in range(epochs):
        print(f"======== Epoch {epoch_i + 1} / {epochs} ========")
        print('Training...')

        t0 = time.time()

        total_train_loss = 0

        for batch in tqdm(train_dataset):
            with tf.GradientTape() as tape:
                input, labels = batch
                input_ids = input['input_ids']
                attention_mask = input['attention_mask']

                embeddings = model((input_ids, attention_mask), training=True)
                
                logits = embeddings.logits
        
                # Calculate loss using the logits
                loss = loss_fn(labels, logits)
                metrics = metrics_fn(labels, logits)

            train_loss(loss)
            train_accuracy(metrics)
            
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))            

        avg_train_loss = train_loss.result()
        avg_train_accuracy = train_accuracy.result()
        training_time = format_time(time.time() - t0)

        print(f"  Average training loss: {avg_train_loss:.5f}")
        print(f"  Average training accuracy: {avg_train_accuracy:.5f}")
        print(f"  Training epoch took: {training_time}")

        # ========================================
        #               Validation
        # ========================================
        print("Running Validation...")

        t0 = time.time()

        total_val_loss = 0


        for batch in tqdm(eval_dataset):
            input, labels = batch
            input_ids = input['input_ids']
            attention_mask = input['attention_mask']

            embeddings = model((input_ids, attention_mask), training=True)
                
            logits = embeddings.logits
    
            # Calculate loss using the logits
            loss = loss_fn(labels, logits)
            metrics = metrics_fn(labels, logits)

            val_loss(loss)
            val_accuracy(metrics)

        avg_val_loss = val_loss.result()
        avg_val_accuracy = val_accuracy.result()
        validation_time = format_time(time.time() - t0)

        print(f"  Validation Loss: {avg_val_loss:.5f}")
        print(f"  Validation Accuracy: {avg_val_accuracy:.5f}")
        print(f"  Validation took: {validation_time}")

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss.numpy(),
                'Valid. Loss': avg_val_loss.numpy(),
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("Training complete!")
    print(f"Total training took {format_time(time.time() - total_t0)} (h:mm:ss)")

    return model, training_stats

In [19]:
model, training_stats = train()

======== Epoch 1 / 5 ========
Training...


100%|██████████| 342/342 [07:54<00:00,  1.39s/it]


  Average training loss: 0.24711
  Average training accuracy: 0.88149
  Training epoch took: 0:07:55
Running Validation...


100%|██████████| 40/40 [00:09<00:00,  4.20it/s]


  Validation Loss: 0.21070
  Validation Accuracy: 0.91305
  Validation took: 0:00:10
======== Epoch 2 / 5 ========
Training...


...

  Validation Loss: 0.21917
  Validation Accuracy: 0.93690
  Validation took: 0:00:10
======== Epoch 5 / 5 ========
Training...


100%|██████████| 342/342 [07:47<00:00,  1.37s/it]


  Average training loss: 0.11011
  Average training accuracy: 0.93134
  Training epoch took: 0:07:48
Running Validation...


100%|██████████| 40/40 [00:09<00:00,  4.22it/s]

  Validation Loss: 0.23667
  Validation Accuracy: 0.94136
  Validation took: 0:00:09
Training complete!
Total training took 0:40:01 (h:mm:ss)


## Test model on new sentences

In [23]:
def predict(text):
    # pre-process text
    encoded_text = tokenizer.encode(text)

    input_ = tf.expand_dims(encoded_text, 0)

    logits = model(input_)[0][0]
    pred = tf.nn.softmax(logits).numpy()
    
    return pred

In [59]:
def predict(text):
    
    sentiment_mapping = {
    "positive": 1,
    "negative": 0,
    "neutral": 2
    }
    
    inputs = tokenizer(
        text,
        return_tensors="tf",
        truncation=True,
        padding="max_length", 
        max_length=128         
    )
    
    #prediksi menggunakan model
    outputs = model(inputs)
    
    logits = outputs.logits
    
    #probabilitas 
    probabilities = tf.nn.softmax(logits).numpy()
    
    #indeks label prediksi
    predicted_index = int(tf.argmax(probabilities, axis=1).numpy()[0])  # Konversi ke integer
    index_to_sentiment = {v: k for k, v in sentiment_mapping.items()}
    predicted_label = index_to_sentiment.get(predicted_index, "unknown")
    
    #keyakinan hasil prediksi
    confidence = probabilities[0][predicted_index]

    print(f"Teks: {text}")
    print(f"Prediksi label: {predicted_label} (Confidence: {confidence:.2f})")

In [70]:
text = "aku sedang jalan-jalan di yogyakarta"

In [71]:
predict(text)

Teks: aku sedang jalan-jalan di yogyakarta
Prediksi label: neutral (Confidence: 1.00)


## Save model


In [20]:
save_path = "finetuned-model"

In [21]:
os.mkdir(save_path)

In [22]:
model.save_pretrained(save_path)

In [72]:
save_directory = "./tokenizer_directory"  # Specify the directory to save the tokenizer
tokenizer.save_pretrained(save_directory)

('./tokenizer_directory/tokenizer_config.json',
 './tokenizer_directory/special_tokens_map.json',
 './tokenizer_directory/vocab.txt',
 './tokenizer_directory/added_tokens.json',
 './tokenizer_directory/tokenizer.json')